In [ ]:
import random
import numpy as np

# City coordinates
cities = {'A': (100, 300), 'B': (200, 130), 'C': (300, 500), 'D': (500, 390),
          'E': (700, 300), 'F': (900, 600), 'G': (800, 950), 'H': (600, 560),
          'I': (350, 550), 'J': (270, 350)}

# Function to calculate Euclidean distance between two cities
def distance(city1, city2):
    return np.linalg.norm(np.array(city1) - np.array(city2))

# Fitness function: total distance of the tour
def fitness(tour):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distance(cities[tour[i]], cities[tour[i+1]])
    total_distance += distance(cities[tour[-1]], cities[tour[0]])  # Return to starting point
    return total_distance

# Create initial population of tours (randomly shuffled cities)
def create_population(pop_size, city_list):
    population = []
    for _ in range(pop_size):
        tour = city_list[:]
        random.shuffle(tour)
        population.append(tour)
    return population

# Selection: choose the best tours
def selection(population, fitnesses, num_best):
    fitness_pairs = list(zip(population, fitnesses))
    fitness_pairs.sort(key=lambda x: x[1])  # Sort by fitness (total distance)
    selected = [tour for tour, fitness in fitness_pairs[:num_best]]
    return selected

# Crossover: combine two tours to create a child tour
def crossover(parent1, parent2):
    start = random.randint(0, len(parent1) - 2)
    end = random.randint(start + 1, len(parent1) - 1)
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]
    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene
    return child

# Mutation: randomly swap two cities in the tour
def mutate(tour, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
    return tour

# Main Genetic Algorithm
def genetic_algorithm(cities, pop_size=100, generations=500, mutation_rate=0.01):
    city_list = list(cities.keys())
    population = create_population(pop_size, city_list)
    for gen in range(generations):
        fitnesses = [fitness(tour) for tour in population]
        selected = selection(population, fitnesses, num_best=pop_size // 2)
        new_population = []
        for i in range(0, len(selected) - 1, 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1 = mutate(crossover(parent1, parent2), mutation_rate)
            child2 = mutate(crossover(parent2, parent1), mutation_rate)
            new_population.extend([child1, child2])
        population = new_population
    best_tour = min(population, key=lambda tour: fitness(tour))
    return best_tour, fitness(best_tour)

# Solve the TSP
best_tour, shortest_distance = genetic_algorithm(cities)
print(f"Shortest distance: {shortest_distance:.2f} miles")
print(f"Sequence order: {' -> '.join(best_tour)} -> A")


Shortest distance: 3114.42 miles
Sequence order: D -> I -> J -> B -> A -> C -> E -> G -> F -> H -> A
